In [126]:
data = """x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj"""

In [127]:
with open("AoC_D24_data.txt", "r") as f:
    data = f.read()

In [128]:
layer0 = {l.split(": ")[0]:int(l.split(": ")[1]) for l in data.split("\n\n")[0].split("\n")}
gates = {l.split(" ")[4]:{"in":[l.split(" ")[0], l.split(" ")[2]], "op":l.split(" ")[1]} for l in  data.split("\n\n")[1].split("\n")}
print(layer0)

{'x00': 1, 'x01': 1, 'x02': 0, 'x03': 0, 'x04': 0, 'x05': 1, 'x06': 0, 'x07': 1, 'x08': 1, 'x09': 0, 'x10': 1, 'x11': 0, 'x12': 0, 'x13': 0, 'x14': 1, 'x15': 1, 'x16': 1, 'x17': 0, 'x18': 1, 'x19': 1, 'x20': 0, 'x21': 0, 'x22': 0, 'x23': 0, 'x24': 1, 'x25': 1, 'x26': 0, 'x27': 0, 'x28': 1, 'x29': 1, 'x30': 0, 'x31': 1, 'x32': 0, 'x33': 0, 'x34': 1, 'x35': 1, 'x36': 0, 'x37': 0, 'x38': 0, 'x39': 0, 'x40': 0, 'x41': 0, 'x42': 0, 'x43': 1, 'x44': 1, 'y00': 1, 'y01': 0, 'y02': 1, 'y03': 1, 'y04': 0, 'y05': 0, 'y06': 1, 'y07': 1, 'y08': 0, 'y09': 1, 'y10': 1, 'y11': 1, 'y12': 1, 'y13': 0, 'y14': 1, 'y15': 0, 'y16': 0, 'y17': 1, 'y18': 1, 'y19': 0, 'y20': 1, 'y21': 1, 'y22': 1, 'y23': 1, 'y24': 0, 'y25': 0, 'y26': 0, 'y27': 0, 'y28': 1, 'y29': 1, 'y30': 0, 'y31': 1, 'y32': 1, 'y33': 0, 'y34': 0, 'y35': 1, 'y36': 0, 'y37': 0, 'y38': 0, 'y39': 1, 'y40': 0, 'y41': 0, 'y42': 1, 'y43': 0, 'y44': 1}


In [146]:
def run_gate(gate, inputs):
    if gate['op'] == "OR":
        return int((inputs[0] == 1) | (inputs[1] == 1))
    elif gate['op'] == "AND":
        return int((inputs[0] == 1) & (inputs[1] == 1))
    elif gate['op'] == "XOR":
        return int((inputs[0] == 1) ^ (inputs[1] == 1))

def run_calculation(layer0, gates):
    activated_layers = layer0.copy()
    gate_activated = set()
    len_gate_activated = -1
    # while len(gates)>len(gate_activated):
    while len(gate_activated) > len_gate_activated:
        len_gate_activated = len(gate_activated)
        for out, g in gates.items():
            if (g['in'][0] in activated_layers.keys()) &(g['in'][1] in activated_layers.keys()):
                activated_layers[out] = run_gate(g, [activated_layers[g['in'][0]],activated_layers[g['in'][1]]])
                gate_activated.add(out)
    if len(gates)>len(gate_activated):
        return None, None
    return int("".join([str(activated_layers[k]) for k in sorted({k:v for k, v in activated_layers.items() if k.startswith('z')}.keys(), reverse=True)]), 2), activated_layers
res, activated_layers = run_calculation(layer0, gates)
res

49574189473968

In [147]:
X = int("".join([str(activated_layers[k]) for k in sorted({k:v for k, v in activated_layers.items() if k.startswith('x')}.keys(), reverse=True)]), 2)
Y = int("".join([str(activated_layers[k]) for k in sorted({k:v for k, v in activated_layers.items() if k.startswith('x')}.keys(), reverse=True)]), 2)
true_result = X+Y
true_result

52885645396806

In [148]:
52885645396806^49574189473968

31950361308150

In [161]:
from tqdm import tqdm
import random
gk = sorted(gates.keys())
all_possible_swaps = [(a, b) for idx, a in enumerate(gk) for b in gk[idx + 1:]]
random.shuffle(all_possible_swaps)
print(all_possible_swaps[:5])
swapped_gates = gates.copy()
selected_swaps = set()
diff = abs(true_result^run_calculation(layer0, gates)[0])
print("diff", diff)
for swap in tqdm(all_possible_swaps):
    test_swap = swapped_gates.copy()
    # print('a')
    tmp = test_swap[swap[0]]
    # print('b')
    test_swap[swap[0]] = test_swap[swap[1]] 
    # print('c')
    test_swap[swap[1]] = tmp
    new_res = run_calculation(layer0, test_swap)[0]
    if new_res is not None:
        # print('d')
        if abs(true_result ^ new_res) < diff:
            diff = abs(true_result - new_res)
            # if swap[0] in [s[0] for s in selected_swaps]:
            #     selected_swaps.remove([(s[0], s[1]) for s in selected_swaps if swap[0]==s[0]][0])
            selected_swaps.add(swap)
            swapped_gates = gates.copy()
            for s in selected_swaps:
                tmp = swapped_gates[s[0]]
                swapped_gates[s[0]] = swapped_gates[s[1]] 
                swapped_gates[s[1]] = tmp
            print("diff", diff, swap)
selected_swaps

[('qtf', 'z05'), ('dhf', 'z18'), ('ppq', 'z44'), ('mjj', 'z33'), ('mrc', 'z38')]
diff 31950361308150


  0%|                                       | 23/24531 [00:00<01:47, 227.92it/s]

diff 3311455922870 ('qtf', 'z05')
diff 1086590522698 ('bhp', 'mjj')


  1%|▎                                     | 189/24531 [00:00<01:00, 403.07it/s]

diff 12652669622 ('ckb', 'z28')


...

 99%|███████████████████████████████████▊| 24406/24531 [00:54<00:00, 458.49it/s]

diff 56 ('z07', 'z08')


100%|████████████████████████████████████| 24531/24531 [00:54<00:00, 451.58it/s]

diff 8 ('ngm', 'z06')


{('bhp', 'mjj'),
 ('ckb', 'z28'),
 ('cmj', 'z32'),
 ('crw', 'pcs'),
 ('ffk', 'nbd'),
 ('gmm', 'pwm'),
 ('hrf', 'z13'),
 ('krf', 'tjd'),
 ('ngm', 'z06'),
 ('pgc', 'z19'),
 ('qnn', 'shd'),
 ('qtf', 'stc'),
 ('qtf', 'z05'),
 ('z07', 'z08')}

In [164]:
52885645396814-52885645396806

8

In [166]:
r, al = run_calculation(layer0, swapped_gates)
r

52885645396814

In [155]:
",".join(sorted([s[0] for s in selected_swaps]) + sorted([s[1] for s in selected_swaps]))

'bhp,cpw,gmm,gsw,mbj,mrc,pfv,qrm,sfj,stc,z18,hdn,jwh,pwm,vmh,z10,z12,z19,z24,z25,z32,z39'

In [160]:
swapped_gates

{'z14': {'in': ['fjm', 'gqp'], 'op': 'XOR'},
 'hdn': {'in': ['x18', 'y18'], 'op': 'XOR'},
 'shd': {'in': ['wbb', 'vnp'], 'op': 'AND'},
 'cqw': {'in': ['srq', 'mpk'], 'op': 'OR'},
 'tjw': {'in': ['y30', 'x30'], 'op': 'AND'},
 'qhf': {'in': ['x26', 'y26'], 'op': 'AND'},
 'kbs': {'in': ['y10', 'x10'], 'op': 'XOR'},
 'swn': {'in': ['y43', 'x43'], 'op': 'XOR'},
 'rkg': {'in': ['sfj', 'jks'], 'op': 'OR'},
 'tct': {'in': ['y01', 'x01'], 'op': 'XOR'},
 'z04': {'in': ['nsv', 'pjt'], 'op': 'XOR'},
 'hfc': {'in': ['dcq', 'knt'], 'op': 'AND'},
 'jks': {'in': ['hfh', 'cjm'], 'op': 'AND'},
 'ngm': {'in': ['hwv', 'cpr'], 'op': 'OR'},
 'nhn': {'in': ['fsm', 'btg'], 'op': 'AND'},
 'tqh': {'in': ['y16', 'x16'], 'op': 'AND'},
 'ksv': {'in': ['qtf', 'nsp'], 'op': 'XOR'},
 'pbj': {'in': ['vjv', 'vvn'], 'op': 'AND'},
 'bpn': {'in': ['y23', 'x23'], 'op': 'XOR'},
 'z20': {'in': ['tsm', 'dnc'], 'op': 'OR'},
 'z19': {'in': ['knt', 'dcq'], 'op': 'XOR'},
 'qqf': {'in': ['ktj', 'cmb'], 'op': 'AND'},
 'cmb': {'in':

In [158]:
gk = sorted(swapped_gates.keys())
all_possible_swaps = [(a, b) for idx, a in enumerate(gk) for b in gk[idx + 1:]]
random.shuffle(all_possible_swaps)
print(all_possible_swaps[:5])
swapped_gates = swapped_gates.copy()
selected_swaps = set()
diff = abs(true_result^run_calculation(layer0, swapped_gates)[0])
print("diff", diff)
for swap in tqdm(all_possible_swaps):
    test_swap = swapped_gates.copy()
    # print('a')
    tmp = test_swap[swap[0]]
    # print('b')
    test_swap[swap[0]] = test_swap[swap[1]] 
    # print('c')
    test_swap[swap[1]] = tmp
    new_res = run_calculation(layer0, test_swap)[0]
    if new_res is not None:
        # print('d')
        if abs(true_result ^ new_res) < diff:
            diff = abs(true_result - new_res)
            # if swap[0] in [s[0] for s in selected_swaps]:
            #     selected_swaps.remove([(s[0], s[1]) for s in selected_swaps if swap[0]==s[0]][0])
            selected_swaps.add(swap)
            swapped_gates = gates.copy()
            for s in selected_swaps:
                tmp = swapped_gates[s[0]]
                swapped_gates[s[0]] = swapped_gates[s[1]] 
                swapped_gates[s[1]] = tmp
            print("diff", diff, swap)
selected_swaps

[('ckh', 'rqp'), ('nbd', 'z35'), ('qhf', 'rnp'), ('bdh', 'kqt'), ('pvg', 'rkg')]


TypeError: unsupported operand type(s) for ^: 'int' and 'NoneType'

In [167]:
"".join([str(al[k]) for k in sorted({k:v for k, v in al.items() if k.startswith('z')}.keys(), reverse=True)])

'1100000001100101100110000110111000101101001110'

In [144]:
format(52885645396806, 'b')

'1100000001100101100110000110111000101101000110'

In [165]:
# swaping this give difference of of 8 (only 1 bit)
# {('bhp', 'mjj'),
#  ('ckb', 'z28'),
#  ('cmj', 'z32'),
#  ('crw', 'pcs'),
#  ('ffk', 'nbd'),
#  ('gmm', 'pwm'),
#  ('hrf', 'z13'),
#  ('krf', 'tjd'),
#  ('ngm', 'z06'),
#  ('pgc', 'z19'),
#  ('qnn', 'shd'),
#  ('qtf', 'stc'),
#  ('qtf', 'z05'),
#  ('z07', 'z08')}